In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import gzip
from time import sleep
import pickle
import sys

In [2]:
with open('query_dict/id_querry_spelled.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [5]:
len(all_groups['DocumentId'].unique())

548594

In [6]:
q_ids = all_groups['QueryId'].unique()
with gzip.open('preembeddings.gz', 'wb') as imp:
    for q1 in tqdm(q_ids):
        tmp = all_groups[all_groups.QueryId==q1]
        docs = tmp['DocumentId'].values
        for doc1 in docs:
            with gzip.open("doc_collection/{}/{}.gz".format(q1, doc1)) as file:
                line_exists = 0
                for line in file:
                    a = line.decode('utf-8', errors = 'ignore').lower().strip('\n').split('\t')
                    line_exists+=1
                    l1 = a[0]+'\n'
                    l1 = l1.encode('utf-8')
                    imp.write(l1)
                if line_exists==0:
                    l1 = '\n'
                    l1 = l1.encode('utf-8')
                    imp.write(l1)

100%|██████████| 6311/6311 [10:00:23<00:00,  5.71s/it]  


In [7]:
#тест числа строк
ind = 0
with gzip.open('preembeddings.gz', 'rb') as imp:
    for line in tqdm(imp):
        ind+=1

606050it [15:57, 632.67it/s] 


In [ ]:
##все ок